In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Tuple

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import numpy as np

In [26]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="bfloat16", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
AutoConfig.from_pretrained(model_name)

Qwen2Config {
  "_name_or_path": "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 131072,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.45.2",
  "use_cache": true,
  "use_mrope": false,
  "use_sliding_window": false,
  "vocab_size": 151936
}

In [62]:
for i in range(5):
    inputs = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": "Can penguins fly? Segment the thinking process into clear steps and indicate \"YES\" or \"NO\" once at the end ."},
        ],
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("mps")
    outputs = model.generate(input_ids=inputs, max_new_tokens=1200, do_sample=False)
    print(tokenizer.decode(outputs[0]))

/Users/bach/anaconda3/envs/arena-env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/bach/anaconda3/envs/arena-env/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexp

<｜begin▁of▁sentence｜><｜User｜>Can penguins fly? Segment the thinking process into clear steps and indicate "YES" or "NO" once at the end .<｜Assistant｜><think>
Okay, so I'm trying to figure out whether penguins can fly. I know penguins are birds, but I'm not exactly sure about their flight capabilities. Let me break this down step by step.

First, I remember that birds can fly, but I'm not sure if penguins fit into that category. I think they're known for being cold-blooded and having a streamlined body shape, which helps them fly. But does that mean they can fly? I'm not certain.

Next, I should consider the physical characteristics of penguins. Penguins have a thick layer of snow cover on their body, which is pretty thick. I think this layer is called a body cover. I've heard that this helps them stay warm, but does it also aid in flight? I'm not sure.

Then, I recall that penguins are adapted to cold environments. They have specialized wings called flippers, which I believe are used f

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Can penguins fly? Segment the thinking process into clear steps and indicate "YES" or "NO" once at the end .<｜Assistant｜><think>
Okay, so I'm trying to figure out whether penguins can fly. I know penguins are birds, but I'm not exactly sure about their flight capabilities. Let me break this down step by step.

First, I remember that birds can fly, but I'm not sure if penguins fit into that category. I think they're known for being cold-blooded and having a streamlined body shape, which helps them fly. But does that mean they can fly? I'm not certain.

Next, I should consider the physical characteristics of penguins. Penguins have a thick layer of snow cover on their body, which is pretty thick. I think this layer is called a body cover. I've heard that this helps them stay warm, but does it also aid in flight? I'm not sure.

Then, I recall that penguins are adapted to cold environments. They have specialized wings called flippers, which I believe are used f

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Can penguins fly? Segment the thinking process into clear steps and indicate "YES" or "NO" once at the end .<｜Assistant｜><think>
Okay, so I'm trying to figure out whether penguins can fly. I know penguins are birds, but I'm not exactly sure about their flight capabilities. Let me break this down step by step.

First, I remember that birds can fly, but I'm not sure if penguins fit into that category. I think they're known for being cold-blooded and having a streamlined body shape, which helps them fly. But does that mean they can fly? I'm not certain.

Next, I should consider the physical characteristics of penguins. Penguins have a thick layer of snow cover on their body, which is pretty thick. I think this layer is called a body cover. I've heard that this helps them stay warm, but does it also aid in flight? I'm not sure.

Then, I recall that penguins are adapted to cold environments. They have specialized wings called flippers, which I believe are used f

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Can penguins fly? Segment the thinking process into clear steps and indicate "YES" or "NO" once at the end .<｜Assistant｜><think>
Okay, so I'm trying to figure out whether penguins can fly. I know penguins are birds, but I'm not exactly sure about their flight capabilities. Let me break this down step by step.

First, I remember that birds can fly, but I'm not sure if penguins fit into that category. I think they're known for being cold-blooded and having a streamlined body shape, which helps them fly. But does that mean they can fly? I'm not certain.

Next, I should consider the physical characteristics of penguins. Penguins have a thick layer of snow cover on their body, which is pretty thick. I think this layer is called a body cover. I've heard that this helps them stay warm, but does it also aid in flight? I'm not sure.

Then, I recall that penguins are adapted to cold environments. They have specialized wings called flippers, which I believe are used f

In [52]:
def gather_residual_activations(model, target_layer, inputs):
    target_act = None
    def gather_target_act_hook(mod, inputs, outputs):
        nonlocal target_act
        target_act = inputs[0]  # Get residual stream from layer input
        return outputs
        
    handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
    with torch.no_grad():
        _ = model(inputs)
    handle.remove()
    return target_act

In [53]:
target_act = gather_residual_activations(model, 11, inputs)

In [54]:
inputs

tensor([[151646, 151644,   6713,    281,  55358,  11466,     30,  37103,    279,
           7274,   1882,   1119,   2797,   7354,    323,  13216,    330,  14004,
              1,    476,    330,   8996,      1,   3055,    518,    279,    835,
            659, 151645, 151648,    198]], device='mps:0')

In [55]:
target_act.shape

torch.Size([1, 31, 1536])

In [56]:
# TODO: is target_act deterministic (same inputs, multiple runs)?

In [57]:
target_act

tensor([[[-3.6562e+00, -2.5312e+00,  3.6000e+01,  ...,  9.4375e+00,
           9.0000e+00,  2.1375e+01],
         [-2.9062e+00, -2.3281e+00,  3.6000e+01,  ...,  9.8125e+00,
           8.3125e+00,  2.1875e+01],
         [ 2.2500e+00,  5.8594e-03, -3.5938e-01,  ..., -2.0312e-01,
           1.1328e+00,  2.5977e-01],
         ...,
         [ 4.5312e+00, -3.9062e-03,  3.9062e-01,  ..., -2.6758e-01,
          -5.5078e-01,  2.3438e+00],
         [ 4.5625e+00,  5.0391e-01,  5.0391e-01,  ...,  4.2969e-02,
           2.8125e-01,  5.8203e-01],
         [ 3.1094e+00, -2.9688e+00,  5.0781e-02,  ...,  1.0703e+00,
           1.9297e+00,  2.0000e+00]]], device='mps:0')

In [58]:
gather_residual_activations(model, 11, inputs)

tensor([[[-3.6562e+00, -2.5312e+00,  3.6000e+01,  ...,  9.4375e+00,
           9.0000e+00,  2.1375e+01],
         [-2.9062e+00, -2.3281e+00,  3.6000e+01,  ...,  9.8125e+00,
           8.3125e+00,  2.1875e+01],
         [ 2.2500e+00,  5.8594e-03, -3.5938e-01,  ..., -2.0312e-01,
           1.1328e+00,  2.5977e-01],
         ...,
         [ 4.5312e+00, -3.9062e-03,  3.9062e-01,  ..., -2.6758e-01,
          -5.5078e-01,  2.3438e+00],
         [ 4.5625e+00,  5.0391e-01,  5.0391e-01,  ...,  4.2969e-02,
           2.8125e-01,  5.8203e-01],
         [ 3.1094e+00, -2.9688e+00,  5.0781e-02,  ...,  1.0703e+00,
           1.9297e+00,  2.0000e+00]]], device='mps:0')